# <center> *ASL DETECTION APP* </center>

# Importing Libraries

In [4]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

# DATA COLLECTION

In [7]:
import cv2
import os

def capture_images(directory='Image2/'):
    # Create directories for A to Z if they don't exist
    for letter in range(ord('A'), ord('Z')+1):
        letter_dir = os.path.join(directory, chr(letter))
        os.makedirs(letter_dir, exist_ok=True)

    cap = cv2.VideoCapture(0)

    try:
        while True:
            _, frame = cap.read()
            rect_width, rect_height = 225,225
            rect_x = (frame.shape[1] - rect_width) // 2
            rect_y = (frame.shape[0] - rect_height) // 2
            cv2.rectangle(frame, (rect_x, rect_y), (rect_x + rect_width, rect_y + rect_height), (0, 255, 0), 2)
            cv2.imshow("data", frame)

            interrupt = cv2.waitKey(10)
            if interrupt != -1:
                key = chr(interrupt & 255)
                if key.isalpha():
                    letter = key.upper()
                    count = len(os.listdir(os.path.join(directory, letter)))
                    try:
                        image_path = os.path.join(directory, letter, f"{count}.png")
                        cv2.imwrite(image_path, frame[rect_y:rect_y+rect_height, rect_x:rect_x+rect_width])
                        print("Saved image in", image_path)
                        feedback_frame = frame.copy()
                        cv2.putText(feedback_frame, f"Captured: {letter}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                        cv2.imshow("data", feedback_frame)
                        cv2.waitKey(100)
                    except Exception as e:
                        print("Error saving image:", e)
            if cv2.getWindowProperty("data", cv2.WND_PROP_VISIBLE) < 1:
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    capture_images()


Saved image in Image2/G\0.png
Saved image in Image2/G\1.png
Saved image in Image2/G\2.png
Saved image in Image2/G\3.png
Saved image in Image2/G\4.png
Saved image in Image2/G\5.png
Saved image in Image2/G\6.png
Saved image in Image2/G\7.png
Saved image in Image2/G\8.png
Saved image in Image2/G\9.png
Saved image in Image2/G\10.png
Saved image in Image2/G\11.png
Saved image in Image2/G\12.png
Saved image in Image2/G\13.png
Saved image in Image2/G\14.png
Saved image in Image2/G\15.png
Saved image in Image2/G\16.png
Saved image in Image2/G\17.png
Saved image in Image2/G\18.png
Saved image in Image2/G\19.png
Saved image in Image2/G\20.png
Saved image in Image2/G\21.png
Saved image in Image2/G\22.png
Saved image in Image2/G\23.png
Saved image in Image2/G\24.png
Saved image in Image2/G\25.png
Saved image in Image2/G\26.png
Saved image in Image2/G\27.png
Saved image in Image2/G\28.png
Saved image in Image2/G\29.png
Saved image in Image2/G\30.png
Saved image in Image2/G\31.png
Saved image in Ima

# making all classes have same number of data

In [2]:
import os

def remove_extra_files(directory, target_count):
    class_directories = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
    
    for d in class_directories:
        dir_path = os.path.join(directory, d)
        files = os.listdir(dir_path)
        current_count = len(files)
        if current_count > target_count:
            extra_count = current_count - target_count
            extra_files = files[-extra_count:]
            for file in extra_files:
                file_path = os.path.join(dir_path, file)
                os.remove(file_path)
                print(f"Removed {file_path}")

# Example usage:
directory_path = 'Image2'
target_count = 1500
remove_extra_files(directory_path, target_count)


Removed Image2\A\701.png
Removed Image2\A\702.png
Removed Image2\A\703.png
Removed Image2\A\704.png
Removed Image2\A\705.png
Removed Image2\A\706.png
Removed Image2\A\707.png
Removed Image2\A\708.png
Removed Image2\A\709.png
Removed Image2\A\71.png
Removed Image2\A\710.png
Removed Image2\A\711.png
Removed Image2\A\712.png
Removed Image2\A\713.png
Removed Image2\A\714.png
Removed Image2\A\715.png
Removed Image2\A\716.png
Removed Image2\A\717.png
Removed Image2\A\718.png
Removed Image2\A\719.png
Removed Image2\A\72.png
Removed Image2\A\720.png
Removed Image2\A\721.png
Removed Image2\A\722.png
Removed Image2\A\723.png
Removed Image2\A\724.png
Removed Image2\A\725.png
Removed Image2\A\726.png
Removed Image2\A\727.png
Removed Image2\A\728.png
Removed Image2\A\729.png
Removed Image2\A\73.png
Removed Image2\A\730.png
Removed Image2\A\731.png
Removed Image2\A\732.png
Removed Image2\A\733.png
Removed Image2\A\734.png
Removed Image2\A\735.png
Removed Image2\A\736.png
Removed Image2\A\737.png
Rem

# Data Preprocessing 

In [1]:
import os
import numpy as np
import cv2
import mediapipe as mp
import logging
import matplotlib.pyplot as plt
import pickle
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
# Initialize MediaPipe Hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.3)
data_dir = "Image2/"
data=[]
lables=[]

for dir_ in os.listdir(data_dir):
    for file in os.listdir(os.path.join(data_dir, dir_)):
        data_aux=[]
    # Read the image
        frame = cv2.imread(os.path.join(data_dir, dir_, file))
        if frame is None:
            logger.error(f"Failed to read image {os.path.join(data_dir, dir_, file)}")
            continue
        # Convert to RGB for MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)   
        results = hands.process(frame_rgb)
        # Check if hands were detected
        if results.multi_hand_landmarks:
            # Get landmarks of the first detected hand
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks on the frame
                mp_drawing.draw_landmarks(
                    frame_rgb, 
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(), 
                    mp_drawing_styles.get_default_hand_connections_style())
                for i in range(len(hand_landmarks.landmark)):
                    x=hand_landmarks.landmark[i].x 
                    y=hand_landmarks.landmark[i].y
                    data_aux.append(x)
                    data_aux.append(y)
            data.append(data_aux)
            lables.append(dir_)          
        else:
            logger.warning(f"Image {os.path.join(data_dir, dir_, file)} - No hand detected")
f=open("data.pickle",'wb')
pickle.dump({"data":data,"lables":lables},f)
f.close()
print("Data saved in data.pickle")

Data saved in data.pickle


# Model Creation

In [2]:
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
data_dict = pickle.load(open("data.pickle", "rb"))
data=np.asarray(data_dict["data"])
lables=np.asarray(data_dict["lables"])
x_train, x_test, y_train, y_test = train_test_split(data, lables, test_size=0.2,shuffle=True,stratify=lables)
model=RandomForestClassifier()
model.fit(x_train,y_train)

f=open("model.p",'wb')
pickle.dump({"model":model},f)
f.close()


# Model Evaluation

In [3]:
y_pred=model.predict(x_test)
print("Accuracy: ",accuracy_score(y_test,y_pred)*100)

Accuracy:  100.0


In [ ]:
import cv2
import mediapipe as mp
import pickle
import numpy as np

cap1 = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.3)

model_dict = pickle.load(open("Frontend Applicaiton\model.p", "rb"))
model = model_dict["model"]
label_dict={0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'I',9:'J',10:'K',11:'L',12:'M',13:'N',14:'O',15:'P',16:'Q',17:'R',18:'S',19:'T',20:'U',21:'V',22:'W',23:'X',24:'Y',25:'Z'}
while True:
    ret, frame = cap1.read()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame, 
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(), 
                mp_drawing_styles.get_default_hand_connections_style()
            )

            data_aux = [coord for landmark in hand_landmarks.landmark for coord in (landmark.x, landmark.y)]
            
            prediction = model.predict_proba([np.asarray(data_aux)])  # Change this line
            predicted_index = np.argmax(prediction[0])
            predicted_label = label_dict[predicted_index]
            accuracy = prediction[0][predicted_index]
            # Draw bounding box
            h, w, _ = frame.shape
            min_x = min(landmark.x for landmark in hand_landmarks.landmark) * w
            max_x = max(landmark.x for landmark in hand_landmarks.landmark) * w
            min_y = min(landmark.y for landmark in hand_landmarks.landmark) * h
            max_y = max(landmark.y for landmark in hand_landmarks.landmark) * h
            cv2.rectangle(frame, (int(min_x), int(min_y)), (int(max_x), int(max_y)), (0, 255, 0), 2)

            # Display label and accuracy
            accuracy_text = f'{accuracy * 100:.2f}%'
            label_text = f'Label: {predicted_label}, Accuracy: {accuracy_text}'
            cv2.putText(frame, label_text, (10, h - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Add a condition to break the loop
        break

cap1.release()
cv2.destroyAllWindows()